In [2]:
#Imports
import pandas as pd
import sklearn as skl
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [17]:
#Read in Data
red_df = pd.read_csv('../Resources/winequality-red.csv', delimiter=';')
white_df = pd.read_csv('../Resources/winequality-white.csv', delimiter=';')

In [4]:
def TestModel(features_to_test):
    #Splitting
    target_red = pd.get_dummies(red_df['quality'])
    features_red = red_df[features_to_test]

    target_white = pd.get_dummies(white_df['quality'])
    features_white = white_df[features_to_test]

    X_train_red, X_test_red, y_train_red, y_test_red = train_test_split(features_red, target_red, random_state=42)
    
    X_train_white, X_test_white, y_train_white, y_test_white = train_test_split(features_white, target_white, random_state=42)


    #Scaling
    X_scaler_red = skl.preprocessing.StandardScaler()
    X_scaler_white = skl.preprocessing.StandardScaler()

    X_scaler_red.fit(X_train_red)
    X_scaler_white.fit(X_train_white)

    X_train_scaled_red = X_scaler_red.transform(X_train_red)
    X_test_scaled_red = X_scaler_red.transform(X_test_red)

    X_train_scaled_white = X_scaler_white.transform(X_train_white)
    X_test_scaled_white = X_scaler_white.transform(X_test_white)


    inputs = len(features_to_test)
    
    #Create and Evaluate Red Model
    model_red = tf.keras.models.Sequential()
    model_red.add(tf.keras.layers.Dense(units=25, activation='selu', input_dim=inputs))
    model_red.add(tf.keras.layers.Dense(units=20, activation='softplus'))
    model_red.add(tf.keras.layers.Dense(units=10, activation='softsign'))
    model_red.add(tf.keras.layers.Dense(len(set(red_df['quality'].values)), activation='softmax'))
    
    model_red.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    fit_model_red = model_red.fit(X_train_scaled_red, y_train_red, epochs=30)
    model_red_loss, model_red_accuracy = model_red.evaluate(X_test_scaled_red, y_test_red, verbose=2)

    #Create and Evaluate White Model
    model_white = tf.keras.models.Sequential()
    model_white.add(tf.keras.layers.Dense(units=25, activation='selu', input_dim=inputs))
    model_white.add(tf.keras.layers.Dense(units=20, activation='softplus'))
    model_white.add(tf.keras.layers.Dense(units=10, activation='softsign'))
    model_white.add(tf.keras.layers.Dense(len(set(white_df['quality'].values)), activation='softmax'))

    model_white.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    fit_model_white = model_white.fit(X_train_scaled_white, y_train_white, epochs=25)
    model_white_loss, model_white_accuracy = model_white.evaluate(X_test_scaled_white, y_test_white, verbose=2)

    return({'Red_Loss':model_red_loss, 'Red_Acc':model_red_accuracy, 
            'White_Loss':model_white_loss, 'White_Acc':model_white_accuracy})
    

In [5]:
feature_results = []
missing_feature_results = []

In [ ]:
#Test Single Features
cols = red_df.drop(columns='quality').columns
results = []
for feature in cols:
    result = TestModel([feature])
    results.append({'feature':feature, 'result':result})
feature_results.append({'single_features':results})

In [8]:
#Test Double Features
cols = red_df.drop(columns='quality').columns
results = []
for feature_a in cols:
    for feature_b in cols:
        result = TestModel([feature_a, feature_b])
        results.append({'features': [feature_a, feature_b], 'result':result})
feature_results.append({'double_features':results})

Epoch 1/30
38/38 [==============================] - 2s 3ms/step - loss: 1.7666 - accuracy: 0.2302
Epoch 2/30
38/38 [==============================] - 0s 4ms/step - loss: 1.3533 - accuracy: 0.3912
Epoch 3/30
38/38 [==============================] - 0s 2ms/step - loss: 1.2607 - accuracy: 0.3895
Epoch 4/30
38/38 [==============================] - 0s 12ms/step - loss: 1.2258 - accuracy: 0.3878
Epoch 5/30
38/38 [==============================] - 0s 2ms/step - loss: 1.2076 - accuracy: 0.4120
Epoch 6/30
38/38 [==============================] - 0s 2ms/step - loss: 1.1974 - accuracy: 0.4470
Epoch 7/30
38/38 [==============================] - 0s 3ms/step - loss: 1.1921 - accuracy: 0.4404
Epoch 8/30
38/38 [==============================] - 0s 3ms/step - loss: 1.1883 - accuracy: 0.4671
Epoch 9/30
38/38 [==============================] - 0s 3ms/step - loss: 1.1841 - accuracy: 0.4637
Epoch 10/30
38/38 [==============================] - 0s 2ms/step - loss: 1.1834 - accuracy: 0.4612
Epoch 11/30
38/38 

In [27]:
#Test Single Missing Features
cols = red_df.drop(columns='quality').columns
results = []
for feature in cols:
    feature_list = list(cols)
    feature_list.remove(feature)
    result = TestModel(feature_list)
    results.append({'feature':feature, 'result':result})
missing_feature_results.append({'single_features':results})

Epoch 1/30
38/38 [==============================] - 1s 3ms/step - loss: 1.8197 - accuracy: 0.0859
Epoch 2/30
38/38 [==============================] - 0s 3ms/step - loss: 1.4239 - accuracy: 0.5171
Epoch 3/30
38/38 [==============================] - 0s 3ms/step - loss: 1.2100 - accuracy: 0.5805
Epoch 4/30
38/38 [==============================] - 0s 3ms/step - loss: 1.1182 - accuracy: 0.5663
Epoch 5/30
38/38 [==============================] - 0s 3ms/step - loss: 1.0691 - accuracy: 0.5863
Epoch 6/30
38/38 [==============================] - 0s 2ms/step - loss: 1.0345 - accuracy: 0.5947
Epoch 7/30
38/38 [==============================] - 0s 3ms/step - loss: 1.0105 - accuracy: 0.5988
Epoch 8/30
38/38 [==============================] - 0s 2ms/step - loss: 0.9944 - accuracy: 0.5980
Epoch 9/30
38/38 [==============================] - 0s 3ms/step - loss: 0.9824 - accuracy: 0.5980
Epoch 10/30
38/38 [==============================] - 0s 2ms/step - loss: 0.9749 - accuracy: 0.6155
Epoch 11/30
38/38 [

In [30]:
#Test Double Missing Features
cols = red_df.drop(columns='quality').columns
results = []
for feature_a in cols:
    for feature_b in cols:
        if feature_a != feature_b:
            feature_list = list(cols)
            feature_list.remove(feature_a)
            feature_list.remove(feature_b)
            result = TestModel(feature_list)
            results.append({'features': [feature_a, feature_b], 'result':result})
missing_feature_results.append({'double_features':results})

Epoch 1/30
38/38 [==============================] - 2s 3ms/step - loss: 1.6649 - accuracy: 0.3261
Epoch 2/30
38/38 [==============================] - 0s 3ms/step - loss: 1.3399 - accuracy: 0.5179
Epoch 3/30
38/38 [==============================] - 0s 4ms/step - loss: 1.2271 - accuracy: 0.5571
Epoch 4/30
38/38 [==============================] - 0s 3ms/step - loss: 1.1661 - accuracy: 0.5621
Epoch 5/30
38/38 [==============================] - 0s 3ms/step - loss: 1.1248 - accuracy: 0.5771
Epoch 6/30
38/38 [==============================] - 0s 3ms/step - loss: 1.0906 - accuracy: 0.5805
Epoch 7/30
38/38 [==============================] - 0s 3ms/step - loss: 1.0649 - accuracy: 0.5838
Epoch 8/30
38/38 [==============================] - 0s 3ms/step - loss: 1.0456 - accuracy: 0.5880
Epoch 9/30
38/38 [==============================] - 0s 3ms/step - loss: 1.0287 - accuracy: 0.5830
Epoch 10/30
38/38 [==============================] - 0s 2ms/step - loss: 1.0150 - accuracy: 0.5830
Epoch 11/30
38/38 [

In [31]:
import json
with open('Outputs/feature_results.json', 'w', encoding='utf-8') as f:
    json.dump(feature_results, f, ensure_ascii=False, indent=4)
with open('Outputs/missing_feature_results.json', 'w', encoding='utf-8') as f:
    json.dump(missing_feature_results, f, ensure_ascii=False, indent=4)